# Guide: How to Train a ProSoRo

This notebook is a guide for training a proprioceptive soft robot (ProSoRo). The ProSoRo's proprioception ability is based on a multimodal variational autoencoder (MVAE) trained with finite element analysis (FEA) data, which includes the shape reconstruction and force estimation. In the following sections, we will set up the environment, prepare the Abaqus simulation dataset, train the MVAE, and finally test the performance of the ProSoRo.

## Environment Setup

Before starting, you should ensure that you have installed `pytorch>=2.1`, and `abaqus>=2022`. 

Then install the required packages by running the following command:

```bash
pip install -r requirements.txt
```

## Input File Template

The `.inp` file is necessary for Abaqus simulation. It contains the information about the geometry, material properties, boundary conditions, history and field outputs, etc. Here we provide typical steps to create an `.inp` file, taking a cylinder as an example. If you don't want to create the `.inp` file yourself, just use the provided `template.inp` in the `./templates/cylinder/` folder.

### Part

After opening the Abaqus CAE, we create a new model database with standard/explicit model. Then a new part should be created by either creating a new `Part` or importing a geometry file. Here we create a new part by selecting `Part` -> `Create` -> `3D` -> `Deformable` -> `Solid` -> `Extrusion` -> `Continue`. Then we create a circle with a radius of 20, and exit the sketch by clicking `Done`. After that, we extrude the circle with a depth of 40 by clicking `OK`. Now we have a cylinder part. It's optional to add some wires on the surface in order to make the mesh finer.

![part.jpg](./assets/img/abaqus/part.jpg)

### Property

In the `Property` module, we create a new `Material`, select the material behavior, and input the corresponding parameters. Here we select `Mechanical` -> `Elasticity` -> `Elastic`, and input `Young's modulus=0.2` and `Poisson's ratio=0.4`. Then we create a new `Section` with `Solid` -> `Homogeneous` type, select the `Material` we just created. After that, we need to assign the `Section` to the part by selecting the part and clicking `Assign Section`. Now the material properties are defined, and the color of the part becomes green.

![property.jpg](./assets/img/abaqus/property.jpg)

### Assembly

In the `Assembly` module, we add the part to the assembly by selecting `Instance` -> `Create` -> `Part` -> `OK`. Note that we set `Instance Type` as `Dependent (mesh on part)`. The axis origin is at the center of the bottom surface of the cylinder, which is convenient for the following steps. And now we need to define sets, reference points, and surfaces, which can be found in the `Tools`:

- `RP-1`: create a reference point at the center of the top surface.
- `Set-RP-1`: create a set and select the reference point `RP-1`.
- `Set-bottom_surface`: create a set and select the bottom surface of the cylinder.
- `Set-top_surface`: create a set and select the top surface of the cylinder.
- `Set-surface`: create a set and select all surfaces of the cylinder.
- `Surf-bottom_surface`: create a surface and select the bottom surface of the cylinder.
- `Surf-top_surface`: create a surface and select the top surface of the cylinder.

![assembly.jpg](./assets/img/abaqus/assembly.jpg)

### Step

In `Step` module, we create a new `Step` with `Static, General` type, and set `Nlgeom` as `On`. Then we need to edit the `Field Output` and select `S, U` as the output variables. For the `History Output`, the domain needs to be selected as `Integrated output section` which needs to be created with `Surf-bottom_surface`, and the output variables are `SOF, SOM`. Since we mainly consider the geometry and force information, other output variables can be ignored. If you are interested in other variables, you can add them as well.

![step.jpg](./assets/img/abaqus/step.jpg)

### Interaction

In the `Interaction` module, we need to define the constraints between `RP-1` and `Surf-top_surface`. `Coupling` type `Constraint` is created with `Set-RP-1` selected as `Control points` and `Surf-top_surface` selected as `Surface`. The `Coupling type` is set as `Kinematic`, and the `Constrained degrees of freedom` are set as all. Now we are able to move the `Surf-top_surface` by changing the position of `RP-1`.

![interaction.jpg](./assets/img/abaqus/interaction.jpg)

### Load

In the `Load` module, we first create a `BC-1` with `Symmetry/Antisymmetry/Encastre` type at `Initial` step, and select `Set-bottom_surface` as the `Region` and `ENCASTRE` as the `Boundary condition`. Then we create a `BC-2` with `Displacement/Rotation` type at `Step-1` step, and select `Set-RP-1` as the `Region` and all degrees of freedom with `0` or other values. 

![load.jpg](./assets/img/abaqus/load.jpg)

### Mesh

In the `Mesh` module, we need to first select the `Mesh controls` and set the `Element shape` as `Tet`. Then we need to make the element type as `C3D4` by selecting `Mesh` -> `Element type` and changing the `Geometric Order` as `Linear`, and the color of the geometry becomes pink. It's optional to refine the mesh by selecting `Seed part` and setting the `Approximate global size`, `Maximum deviation factor`, and `Minimum size`.

![mesh.jpg](./assets/img/abaqus/mesh.jpg)

### Job

After all above steps, we create the job to generate the `.inp` file. We can select `Create` in `Job manager`, and click the `Data Check` to check whether there are any errors. If there is no error, we can click `Write input` to generate the `.inp` file. The `.inp` file will be saved in the working directory, which is going to be used in the following sections.

![job.jpg](./assets/img/abaqus/job.jpg)

## Setup

Here are some necessary packages and functions needed to be imported.

In [ ]:
import os
import shutil
from typing import List
import csv
import random
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from pytorch_lightning import LightningModule
import matplotlib.pyplot as plt
import plotly.graph_objects as go

## Motion Data

In this section, we are going to generate motion data of the `RP-1` in the `.inp` file. First, we need to define serveral settings:

- `object`: the type of the target module object, which is `cylinder` in this example.
- `data_path`: the path to save the motion data, typically based on the `object`.
- `data_num`: the number of motion data to generate.

In [ ]:
# Module type
object = "cylinder"
# Data path
data_path = "./data/" + object + "/"
if not os.path.exists(data_path):
    os.makedirs(data_path)
# Data number
data_num = 50000

Then we can generate the motion, including `u1`, `u2`, `u3`, `r1`, `r2`, `r3`. Each component is randomly generated within a certain range. You can adjust the range based on your requirements.

In [ ]:
# Generate motion.csv
csv_file = open(data_path + "motion.csv", "w", encoding="utf-8", newline="")
csv_writer = csv.writer(csv_file)
count = 0
while count < data_num:
    u1 = random.uniform(-8, 8)
    u2 = random.uniform(-8, 8)
    u3 = random.uniform(-3, 3)
    ur1 = random.uniform(0, 0.3) * (1 if u2 < 0 else -1)
    ur2 = random.uniform(0, 0.3) * (1 if u1 > 0 else -1)
    ur3 = random.uniform(-0.3, 0.3)
    count += 1
    csv_writer.writerow(
        [
            count,
            np.around(u1, 3),
            np.around(u2, 3),
            np.around(u3, 3),
            np.around(ur1, 2),
            np.around(ur2, 2),
            np.around(ur3, 2),
        ]
    )
csv_file.close()
print("Motion.csv has been generated successfully.")

After generating the motion, we can visualize the data points in the 3D space.

In [ ]:
# Visualize motion.csv
csv_file = open(data_path + "motion.csv", "r", encoding="utf-8")
csv_reader = csv.reader(csv_file)
motion = []
for item in csv_reader:
    motion.append(item)
csv_file.close()
motion = np.array(motion[1:]).astype(np.float32)
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.scatter(motion[:, 1], motion[:, 2], motion[:, 3], c="r", marker="o")

## Input File Generation

First, we need to change some configurations in the `.inp` file to make it as a template. We just replace the `BC-2` settings with the following text:

``` text
** Name: BC_2 Type: Displacement/Rotation
*Boundary
Set-RP-1, 1, 1, u1
Set-RP-1, 2, 2, u2
Set-RP-1, 3, 3, u3
Set-RP-1, 4, 4, ur1
Set-RP-1, 5, 5, ur2
Set-RP-1, 6, 6, ur3
```

and also rename the file as `template.inp`, and move it to `./templates/cylinder/`.

To faster the simulation, we can seperate the simulation into multiple parts and run them in parallel. Here we set:

In [ ]:
part_num = 10

Then `.inp` files will be generated into folders. Now just run the following cell to complete the `.inp` file generation.

In [ ]:
# Read template.inp
temp_file = open("./templates/" + object + "/template.inp", "r")
temp_lines = temp_file.read()
temp_file.close()

# Read motion.csv file
motion_list = pd.read_csv(data_path + "motion.csv", header=None)
# Seperate motion_list
for i in range(part_num):
    motion_list.iloc[
        int(len(motion_list) / part_num * i) : int(len(motion_list) / part_num * (i + 1))
    ].to_csv(data_path + "motion_" + str(i + 1) + ".csv", header=None, index=None)

for i in range(1, part_num + 1):
    # Read motion_*.csv file
    motion_csv_name = data_path + "motion_" + str(i) + ".csv"
    motion_csv_file = open(motion_csv_name, "r", encoding="utf-8")
    motion_csv_reader = csv.reader(motion_csv_file)

    # Create folder to store *.inp files
    inp_path = "data/" + object + "/abq_file_" + str(i) + "/"
    if not os.path.exists(inp_path):
        os.makedirs(inp_path)

    # Write *.inp files
    for motion in motion_csv_reader:
        inp_lines = temp_lines
        inp_lines = (
            inp_lines.replace("u1", motion[1])
            .replace("u2", motion[2])
            .replace("u3", motion[3])
            .replace("ur1", motion[4])
            .replace("ur2", motion[5])
            .replace("ur3", motion[6])
        )
        inp_file = open(inp_path + motion[0] + ".inp", "w")
        inp_file.write(inp_lines)
        inp_file.close()

    motion_csv_file.close()
    os.remove(motion_csv_name)

print("All *.inp files have been written successfully.")

## Batch Submission

For Ubuntu (Linux), you can use the following command in `Terminal` to submit a single job:

```bash
abq job={JOB_NAME} cpus=4 int
```

To submit multiple jobs in batch, you first need a `.sh` file with the following content:

```bash
#!/bin/bash

pasway=`dirname $0`
pushd $pasway
echo $basename
for file in `ls $pasway/*.inp`;
do
abq job=`basename $file` cpus=4 int;
done
echo all finished
read -n 1
```

The `.sh` file is already prepared in `./scripts/cmd/` folder. You can run the following command to copy the `.sh` file to the folders.

In [ ]:
for i in range(1, part_num + 1):
    src_file = "./scripts/cmd/run.sh"
    dst_file = "./data/" + object + "/abq_file_" + str(i) + "/run.sh"
    shutil.copy(src_file, dst_file)

Now you can open `Terminal` in each `abq_file_{PART_NUM}` folder and run the following command to start the simulation:

```bash
cd data/cylinder/abq_file_{PART_NUM}
sudo chmod +x run.sh
sh run.sh
```

For windows, you can use the following command in `PowerShell` to submit a single job:

```bash
abaqus job={JOB_NAME} cpus=4 int
```

To submit multiple jobs in batch, you first need a `.bat` file with the following content:

```bat
@echo off
for /f "delims=" %%i in ('dir /b *.inp') do (
abaqus job=%%i cpus=4 int
)
```

The `.bat` file is already prepared in `./scripts/cmd/` folder. You can run the following command to copy the `.bat` file to the folders.

In [ ]:
for i in range(1, part_num + 1):
    src_file = "./scripts/cmd/run.bat"
    dst_file = "./data/" + object + "/abq_file_" + str(i) + "/run.bat"
    shutil.copy(src_file, dst_file)

Now you can open `PowerShell` in each `abq_file_{PART_NUM}` folder and run the following command to start the simulation:

```bash
cd data\cylinder\abq_file_{PART_NUM}
run.bat
```

It will take some time to finish the simulation, so please be patient. All files generated during the simulation will be saved in the `data/cylinder/abq_file/` folder.

## Data Post-processing

After finishing all the simulations, we need to collect data from `.odb` files.

First, we should move all files into `abq_file/` folder.

In [ ]:
# Move all files to one folder
dst_path = data_path + "abq_file/"
if not os.path.exists(dst_path):
    os.makedirs(dst_path)

part_num = 10
for i in range(1, part_num + 1):
    src_path = data_path + "abq_file_" + str(i) + "/"

    for file in os.listdir(src_path):
        shutil.move(src_path + file, dst_path + file)
    os.rmdir(src_path)

print("All files have been moved successfully.")

Then we need to find out which files are completed successfully.

In [ ]:
# Get all .sta file names
abq_path = "./data/" + object + "/abq_file/"
files = os.listdir(abq_path)
stas = []
for file in files:
    if file.endswith(".sta"):
        stas.append(file)
    else:
        pass

# Get all *.odb file names of the jobs that has completed successfully
criteria = [" THE ANALYSIS HAS COMPLETED SUCCESSFULLY\n"]
odbs = []
for sta in stas:
    # Linux: 'utf-8' Windows: 'gbk'
    last_line = open(os.path.join(abq_path, sta), "r", encoding="gbk").readlines()[-1]
    if last_line in criteria:
        odbs.append(sta.replace("sta", "odb"))

csv_path = "data/" + object + "/"
csv_file = open(
    os.path.join(csv_path, "cpl_odb.csv"), "w", encoding="utf-8", newline=""
)
csv_writer = csv.writer(csv_file)
for odb in odbs:
    csv_writer.writerow([odb])
csv_file.close()

print(
    "All *.odb file names of the jobs that has completed successfully have been selected in cpl_odb.csv."
)

A scirpt `read_odb.py` is provided in the `./scripts/` folder to read the `.odb` files and extract the data. We can run the following command to extract the data:

```bash
abaqus cae script=scripts/read_odb.py -- cylinder
```

The data will be saved in a new `csv_file/` folder. Now we can collect the data and save them into `.npy` files.

In [ ]:
# Collect motion, force and shape data from *.csv files
odb_list_file = open(data_path + "cpl_odb.csv", "r")
odb_list = csv.reader(odb_list_file, delimiter=",")
motion = []
force = []
node = []
for odb in odb_list:
    csv_file = open(data_path + "abq_file/" + "".join(odb).replace(".odb", ".csv"), "r")
    csv_reader = csv.reader(csv_file)
    node_rows = []
    for i, rows in enumerate(csv_reader):
        if i == 0:
            motion.append(rows[1:])
        if i == 1:
            force.append(rows)
        if i >= 2:
            node_rows.append(rows)
    node.append(np.array(node_rows, dtype=float))
    csv_file.close()
odb_list_file.close()

# Save data as npy file
node = np.array(node, dtype=float)
print("Shape of node:", node.shape)
motion = np.array(motion, dtype=float)
print("Shape of motion:", motion.shape)
force = np.array(force, dtype=float)
print("Shape of force:", force.shape)
data = np.concatenate(
    (
        motion,
        force,
        node[:, :, 1:].reshape(node.shape[0], node.shape[1] * (node.shape[2] - 1)),
    ),
    axis=1,
)
print("Shape of data:", data.shape)
np.save(data_path + "data.npy", data)

print("The npy files have been written successfully.")

Now we have collected the data into `.npy` files. Then we seperate `data.npy` into `train.npy` and `test.npy` with a ratio of 0.8:0.2.

In [ ]:
# Randomly seperate data into training data and testing data
data = np.load(data_path + "data.npy")
np.random.shuffle(data)
training_data = data[: int(len(data) * 0.8)]
np.save(data_path + "training_data.npy", training_data)
print("Shape of training Data:", training_data.shape)
testing_data = data[int(len(data) * 0.8) :]
np.save(data_path + "testing_data.npy", testing_data)
print("Shape of testing data:", testing_data.shape)

Now we have successfully prepared the dataset for training and testing the MVAE model.

## Training

Now we are going to train the MVAE model with the dataset we prepared in the previous section. `train.py` is the script to train and there are several arguments you can set:

- `--lr`: the learning rate of the optimizer, which is `1e-5` in this example.
- `--batch-size`: the batch size of the training data, which is `128` in this example.
- `--max-epochs`: the maximum epoch of the training process, which is `1000` in this example.

Moreover, there are some parameters for the MVAE model defined in `./config/model.yaml`:

- `object`: the type of the target object, which is `cylinder` in this example.
- `x_dim`: the dimension list of input data x, which is `[6, 6, 2148]` for the cylinder in this example.
- `z_dim`: the dimension of the latent variable, which is `32` in this example.
- `h1_dim`: the dimension list of the first hidden layer, which is `[16, 16, 1024]` in this example.
- `h2_dim`: the dimension list of the second hidden layer, which is `[32, 32, 256]` in this example.
- `recon_pred_scale`: the scale of reconstruction and prediction loss.
- `z_coef`: the coefficient of the latent loss.
- `kl_coef`: the coefficient of the KL divergence.

You can set these parameters based on your requirements in the `train.py` script, and then run the following command to start the training process:

```bash
python train.py
```

The training is based on PyTorch Lightning, which will save the logs in `./lightning_logs/`. The `.pth` file will be saved in `model/pths/`, and named as `mvae_{object}_{recon_pred_scale}_{z_coef}_{kl_coef}_{z_dim}.pth`.

## Testing

After training, you can test the performance of the MVAE model with the test dataset. First, we need to load the trained model and the test dataset:

In [ ]:
# Define the model
class MVAE(LightningModule):
    def __init__(
        self,
        x_dim: list = [6, 6, 2136],
        h1_dim: list = [8, 8, 512],
        h2_dim: list = [16, 16, 128],
        z_dim: int = 32,
        lr: float = 1e-4,
        recon_pred_scale: float = 1,
        z_coef: float = 1,
        kl_coef: float = 1,
        **kwargs,
    ) -> None:
        # Call the super constructor
        super().__init__()

        # Save hyperparameters
        self.save_hyperparameters()
        self.lr = lr
        self.z_dim = z_dim
        self.x_dim = x_dim
        self.h1_dim = h1_dim
        self.h2_dim = h2_dim
        self.recon_pred_scale = recon_pred_scale
        self.z_coef = z_coef
        self.kl_coef = kl_coef

        # Define the model architecture
        for i in range(len(self.x_dim)):
            # Encoder
            setattr(
                self,
                f"encoder_{i}",
                nn.Sequential(
                    nn.Linear(self.x_dim[i], self.h1_dim[i]),
                    nn.ReLU(),
                    nn.Linear(self.h1_dim[i], self.h2_dim[i]),
                ),
            )
            # Decoder
            setattr(
                self,
                f"decoder_{i}",
                nn.Sequential(
                    nn.Linear(self.z_dim, self.h2_dim[i]),
                    nn.ReLU(),
                    nn.Linear(self.h2_dim[i], self.h1_dim[i]),
                    nn.ReLU(),
                    nn.Linear(self.h1_dim[i], self.x_dim[i]),
                ),
            )
            # mu and var
            setattr(self, f"fc_mu_{i}", nn.Linear(self.h2_dim[i], self.z_dim))
            setattr(self, f"fc_var_{i}", nn.Linear(self.h2_dim[i], self.z_dim))

    def sample(self, mu: Tensor, var: Tensor):
        # Calculate the p and q
        std = torch.exp(var / 2)
        p = torch.distributions.Normal(torch.zeros_like(mu), torch.ones_like(std))
        q = torch.distributions.Normal(mu, std)

        # Sample the z
        z = q.rsample()

        # Return the output data
        return p, q, z

    def x_to_z_encoder(self, x: Tensor, input_index: int):
        """Encoder from x to z.

        Args:
            x: the input data.
            input_index: the index of the input data.

        Returns:
            z: the z.
        """

        # Encoder with index
        h = getattr(self, f"encoder_{input_index}")(x)
        # mu and var with index
        mu = getattr(self, f"fc_mu_{input_index}")(h)
        var = getattr(self, f"fc_var_{input_index}")(h)
        # Sample
        _, _, z = self.sample(mu, var)

        # Return z
        return z

    def z_to_x_decoder(self, z: Tensor, output_index: int):
        """Decoder from z to x.

        Args:
            z: the input data.
            output_index: the index of the output data.

        Returns:
            x_hat: the output data.
        """

        # Decoder with index
        x_hat = getattr(self, f"decoder_{output_index}")(z)

        # Return x_hat
        return x_hat

    def forward(self, x_list: List[Tensor]):
        # Create the list to store the output data
        x_hat_list = []

        # Forward the model
        for i in range(len(self.x_dim)):
            # Encoder
            z = self.x_to_z_encoder(x_list[i], i)
            # Decoder
            x_hat = self.z_to_x_decoder(z, i)
            x_hat_list.append(x_hat)

        # Return the output data
        return x_hat_list

    def forward_with_index(self, x: Tensor, input_index: int, output_index: int):
        # Encoder with index
        z = self.x_to_z_encoder(x, input_index)
        # Decoder with index
        x_hat = self.z_to_x_decoder(z, output_index)

        # Return the output data
        return x_hat


# Hyperparameters
object = "cylinder"
recon_pred_scale = 1
z_coef = 1
kl_coef = 0.1
x_dim = [6, 6, 2736]
h1_dim = [16, 16, 1024]
h2_dim = [32, 32, 256]
z_dim = 32

# Load the model
pth_path = (
    "./models/checkpoints/mvae_"
    + object
    + "_"
    + str(recon_pred_scale)
    + "_"
    + str(z_coef)
    + "_"
    + str(kl_coef)
    + "_"
    + str(z_dim)
    + ".pth"
)
model = MVAE(
    x_dim=x_dim,
    h1_dim=h1_dim,
    h2_dim=h2_dim,
    z_dim=z_dim,
    recon_pred_scale=recon_pred_scale,
    z_coef=z_coef,
    kl_coef=kl_coef,
)
model.load_state_dict(torch.load(pth_path))
model.eval()

# Load the data
data_path = "./data/" + object + "/"
mu = np.load(data_path + "mu.npy")
std = np.load(data_path + "std.npy")
testing_data = np.load(data_path + "testing_data.npy")

Now we can test the performance of force estimation and node reconstruction based on the motion data. First, let's predict the force and node displacement:

In [ ]:
# Predict the force and node
motion_list = testing_data[:, :6]
force_gt_list = testing_data[:, 6:12]
node_gt_list = testing_data[:, 12:].reshape(-1, 3)
force_pred_list = []
node_pred_list = []
for i in range(len(testing_data)):
    motion = motion_list[i]
    motion = (motion - mu[:6]) / std[:6]
    motion_tensor = torch.from_numpy(motion).float()
    force_pred = model.forward_with_index(motion_tensor, 0, 1).detach().numpy()
    force_pred = force_pred * std[6:12] + mu[6:12]
    force_pred_list.append(force_pred)
    node_pred = model.forward_with_index(motion_tensor, 0, 2).detach().numpy()
    node_pred = node_pred * std[12:] + mu[12:]
    node_pred_list.append(node_pred)
force_pred_list = np.array(force_pred_list)
node_pred_list = np.array(node_pred_list).reshape(-1, 3)

Based on the predicted data and ground truth, we can calculate the r.s.m.e. (root mean square error) and R2 score:

In [ ]:
# Calculate rmse
force_rmse = np.sqrt(np.mean((force_gt_list[:, :3] - force_pred_list[:, :3]) ** 2))
torque_rmse = np.sqrt(np.mean((force_gt_list[:, 3:] - force_pred_list[:, 3:]) ** 2))
node_rmse = np.sqrt(np.mean((node_gt_list - node_pred_list) ** 2))
print("Force RMSE:", force_rmse)
print("Torque RMSE:", torque_rmse)
print("Node RMSE:", node_rmse)

# Calculate r2
force_r2 = 1 - np.sum((force_gt_list[:, :3] - force_pred_list[:, :3]) ** 2) / np.sum(
    (force_gt_list[:, :3] - np.mean(force_gt_list[:, :3])) ** 2
)
torque_r2 = 1 - np.sum((force_gt_list[:, 3:] - force_pred_list[:, 3:]) ** 2) / np.sum(
    (force_gt_list[:, 3:] - np.mean(force_gt_list[:, 3:])) ** 2
)
node_r2 = 1 - np.sum((node_gt_list - node_pred_list) ** 2) / np.sum(
    (node_gt_list - np.mean(node_gt_list)) ** 2
)
print("Force R2:", force_r2)
print("Torque R2:", torque_r2)
print("Node R2:", node_r2)

We can also plot the predicted and ground truth data to visualize the performance:

In [ ]:
# Linear plot function
def gt_pred_plot(gt, pred, name, color):
    subplot_num = len(gt[0])
    fig = plt.figure(figsize=(3 * subplot_num, 3), dpi=200)
    for i in range(subplot_num):
        plt.subplot(1, subplot_num, i + 1)
        plt.plot(gt[:, i], pred[:, i], ".", color=color, markersize="1")
        plt.plot(
            [0, 1],
            [0, 1],
            "k--",
            linewidth=0.8,
            alpha=0.5,
            transform=plt.gca().transAxes,
        )
        plt.xlabel("Ground truth")
        plt.ylabel("Prediction")
        plt.grid()
        plt.gca().set_aspect("equal", adjustable="box")
    fig.suptitle(name)
    fig.tight_layout(pad=0.4, w_pad=0, h_pad=10)


# Plot the force
random_index = np.random.choice(len(force_gt_list), 2000)
gt_pred_plot(
    gt=force_gt_list[random_index, :3],
    pred=force_pred_list[random_index, :3],
    name="Force",
    color="#38bdf6",
)
gt_pred_plot(
    gt=force_gt_list[random_index, 3:],
    pred=force_pred_list[random_index, 3:],
    name="Torque",
    color="#38bdf6",
)

# Plot the node
random_index = np.random.choice(len(node_gt_list), 2000)
gt_pred_plot(
    gt=node_gt_list[random_index],
    pred=node_pred_list[random_index],
    name="Node",
    color="#38bdf6",
)

Now we can imput a motion and get the predicted force with the following cell:

In [ ]:
motion = np.array([0, 0, 0, 0, 0, 0])
print(
    "Motion:[%.3f, %.3f, %.3f, %.3f, %.3f, %.3f]"
    % (motion[0], motion[1], motion[2], motion[3], motion[4], motion[5])
)
motion_tensor = torch.from_numpy((motion - mu[:6]) / std[:6]).float()
force_pred = model.forward_with_index(motion_tensor, 0, 1).detach().numpy()
force_pred = force_pred * std[6:12] + mu[6:12]
print(
    "Force:[%.3f, %.3f, %.3f, %.3f, %.3f, %.3f]"
    % (
        force_pred[0],
        force_pred[1],
        force_pred[2],
        force_pred[3],
        force_pred[4],
        force_pred[5],
    )
)

It's also able to visualize the shape reconstruction in 3D. First, we need to prepared two files in `templates/cylinder/`:

- `node.txt`: the node coordinates, copied from the `template.inp`.
- `element.txt`: the element nodes, copied from the `template.inp`.

Since the element type is `C3D4`, we can use the following code to generate triangular faces:

In [ ]:
object = "cylinder"
element = np.loadtxt("./templates/" + object + "/element.txt", delimiter=",")
triangles = []
for i in range(element.shape[0]):
    triangles.append([element[i, 1], element[i, 2], element[i, 3]])
    triangles.append([element[i, 1], element[i, 3], element[i, 4]])
    triangles.append([element[i, 1], element[i, 2], element[i, 4]])
    triangles.append([element[i, 2], element[i, 3], element[i, 4]])
triangles = np.sort(np.array(triangles, dtype=int), axis=1)
triangles = np.array(list(set([tuple(t) for t in triangles])), dtype=int)
np.savetxt(
    "templates/" + object + "/triangles.txt", triangles, delimiter=",", fmt="%7d"
)

We also need to prepare the `surf_index.txt` file, which contains the surface index of the cylinder. The surface index can be copied from the `template.inp` file, and saved as `surf_index.txt` in the `templates/cylinder/` folder.

We mainly consider the surface nodes, so we can filter the nodes and triangles:

In [ ]:
node = np.loadtxt("./templates/" + object + "/node.txt", delimiter=",")
surf_index = np.loadtxt(
    "./templates/" + object + "/surf_index.txt", delimiter=","
).astype(int)
surf_node = []
for i in range(node.shape[0]):
    if node[i, 0] in surf_index:
        surf_node.append(node[i])
surf_node = np.array(surf_node)
np.savetxt(
    "./templates/" + object + "/surf_node.txt",
    surf_node,
    delimiter=",",
    fmt="%7d, %12.7f, %12.7f, %12.7f",
)

triangles = np.loadtxt("./templates/" + object + "/triangles.txt", delimiter=",")
surf_index = np.loadtxt(
    "./templates/" + object + "/surf_index.txt", delimiter=","
).astype(int)
surf_triangles = []
for i in range(triangles.shape[0]):
    if (
        triangles[i, 0] in surf_index
        and triangles[i, 1] in surf_index
        and triangles[i, 2] in surf_index
    ):
        surf_triangles.append(triangles[i])
surf_triangles = np.array(surf_triangles)
for i in range(surf_triangles.shape[0]):
    for j in range(3):
        surf_triangles[i, j] = surf_index.tolist().index(surf_triangles[i, j])
np.savetxt(
    "./templates/" + object + "/surf_triangles.txt",
    surf_triangles,
    delimiter=",",
    fmt="%7d, %7d, %7d",
)

Then we initialize the plotly figure and add the cylinder without deformation:

In [ ]:
node_pred = np.zeros_like(surf_node[:, 1:])
node_curr = surf_node.copy()
node_curr[:, 1:] += node_pred

mesh3d = go.Mesh3d(
    x=node_curr[:, 1],
    y=node_curr[:, 2],
    z=node_curr[:, 3],
    i=surf_triangles[:, 0],
    j=surf_triangles[:, 1],
    k=surf_triangles[:, 2],
    colorscale="Viridis",
    autocolorscale=False,
    colorbar=dict(
        title=dict(
            text="Displacement (mm)", side="right", font=dict(size=16, family="Arial")
        ),
        tickvals=[0, 4, 8, 12],
        tickfont=dict(size=14, family="Arial"),
        len=0.5,
    ),
    cmin=0,
    cmax=12,
    intensity=np.linalg.norm(node_pred, axis=1),
    lighting=dict(ambient=1, specular=0, diffuse=0),
    opacity=1,
    showscale=True,
)

fig = go.Figure(data=[mesh3d])
fig.update_layout(
    scene=dict(
        camera=dict(
            eye=dict(
                x=1,
                y=-1,
                z=1,
            ),
            projection=dict(
                type="orthographic",
            ),
        ),
        xaxis=dict(
            nticks=4,
            range=[-50, 50],
        ),
        yaxis=dict(
            nticks=4,
            range=[-50, 50],
        ),
        zaxis=dict(
            nticks=4,
            range=[0, 60],
        ),
        aspectmode="manual",
        aspectratio=go.layout.scene.Aspectratio(
            x=1,
            y=1,
            z=0.75,
        ),
    ),
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=0,
    ),
)
fig.show()

Now we are able to visualize the shape reconstruction of the cylinder with the input motion:

In [ ]:
motion = np.array([8, 8, -1, -0.2, 0.2, 0])

motion_tensor = torch.from_numpy((motion - mu[:6]) / std[:6]).float()
node_pred = model.forward_with_index(motion_tensor, 0, 2).detach().numpy()
node_pred = node_pred * std[12:] + mu[12:]
node_pred = node_pred.reshape(-1, 3)
node_curr = surf_node.copy()
node_curr[:, 1:] += node_pred

fig.update_traces(
    x=node_curr[:, 1],
    y=node_curr[:, 2],
    z=node_curr[:, 3],
    intensity=np.linalg.norm(node_pred, axis=1),
)

If you have made a real ProSoRo by following the [hardware guide](https://sites.google.com/view/prosoro-hardware), you can use modules in `./modules/` folder to visualize it in an interface. There are three modules provided:

- `camera`: capturing the image and estimating the motion of ProSoRo.
- `led`: controlling the LED light of the ProSoRo.
- `interface`: visualizing the image, motion, force, and shape of the ProSoRo.

Start all modules and open the interface in the browser to visualize the ProSoRo. More details can be found in the `./modules/README.md`.